# Advanced layer types: Recurrence

## Questions

- Why do we need layers specifically designed for sequential data?
- What are Recurrent Neural Networks (RNNs) and LSTMs?
- How does an LSTM "remember” important information over time?
- What are alternatives like attention?

## Objectives

- Understand the structure and motivation behind RNN and LSTM layers
- Relate LSTM concepts to earlier architectures (dense, CNN)
- Explore a simple forecasting example using LSTM


## Revisiting sunshine hours

Yesterday, we predicted today's sunshine hours (in Basel) using weather variables from just yesterday — a one-to-one mapping. Each input was a single day's data. Let's rebuild that model quickly to remind ourselves of the test set performance. 

In [ ]:
import pandas as pd
data = pd.read_csv("https://zenodo.org/record/5071376/files/weather_prediction_dataset_light.csv?download=1")

In [ ]:
import pandas as pd

filename_data = "data/weather_prediction_dataset_light.csv"
data = pd.read_csv(filename_data)
data.head()

In [ ]:
# Use only Basel-specific predictors
basel_columns = [col for col in data.columns if col.startswith('BASEL_')]

# Define number of rows (e.g., 9 years of daily data)
nr_rows = 365 * 9

# Drop DATE and MONTH, keep Basel predictors
X_data = data.loc[:nr_rows, basel_columns]
y_data = data.loc[1:(nr_rows + 1), "BASEL_sunshine"]
X_data.shape

In [ ]:
X_data.head()

This time, we'll apply a more appropriate split for a temporal dataset. We'll turn off shuffling so that the test set consists only of later time points than those in the training set. This setup allows us to evaluate how well the model can predict future values using only past information — without accidentally training on data from the future. This is important, because, the future often doesn't resemble the past, and we want to see how well our model can handle this.

In [ ]:
from sklearn.model_selection import train_test_split
test_set_size = 0.2
X_train, X_holdout, y_train, y_holdout = train_test_split(X_data, y_data, test_size=test_set_size, random_state=0, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, test_size=0.5, random_state=0, shuffle=False)
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)


Set seeds to control for random weight initalization

In [ ]:
from tensorflow import keras
from numpy.random import seed
seed(42)
keras.utils.set_random_seed(42)

In [ ]:
from tensorflow import keras

def create_dense_nn(input_shape):
    # Input layer
    inputs = keras.Input(shape=input_shape, name='input')

    # Dense layers
    layers_dense = keras.layers.Dense(100, 'relu')(inputs)
    layers_dense = keras.layers.Dense(50, 'relu')(layers_dense)

    # Output layer
    outputs = keras.layers.Dense(1)(layers_dense)

    return keras.Model(inputs=inputs, outputs=outputs, name="dense_weather_prediction_model")



In [ ]:
model_dense = create_dense_nn(input_shape=(X_data.shape[1],))
model_dense.summary()

In [ ]:
def compile_model(model):
    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=[keras.metrics.RootMeanSquaredError()])


In [ ]:
compile_model(model_dense)

Fit with early stopping, as we did previously

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlystopper = EarlyStopping(
    monitor='val_loss',
    patience=10
    )

history_dense = model_dense.fit(X_train, y_train,
                    batch_size = 32,
                    epochs = 200,
                    validation_data=(X_val, y_val),
                    callbacks=[earlystopper])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_history(history, metrics):
    """
    Plot the training history

    Args:
        history (keras History object that is returned by model.fit())
        metrics (str, list): Metric or a list of metrics to plot
    """
    history_df = pd.DataFrame.from_dict(history.history)
    sns.lineplot(data=history_df[metrics])
    plt.xlabel("epochs")
    plt.ylabel("metric")


In [ ]:
plot_history(history_dense, ['root_mean_squared_error', 'val_root_mean_squared_error'])

In [ ]:
model_dense.evaluate(X_test, y_test)

If you recall, our baseline model had an RMSE of 3.88. Our fully connect neural network only does slightly better than baseline (3.59). While it's encouraging to see an improvement to the baseline, there is much more we can do to improve this result. 

Recall that here, we are only looking at the current day's weather (across cities) to determine sunshine hours in Basel the next day. But what if sunshine patterns depend on the past week, past month, or past year? 

### When is a single lag not enough?

In many real-world tasks, yesterday's data alone isn't sufficient — patterns unfold over time:

- Rainy streaks often last several days
- Cold fronts move gradually, not all at once
- In other domains: heartbeats, language, gestures, and biological sequences like proteins rely on order and context across multiple steps

#### Include mutliple lags manually?
A natural next step is to **include multiple lags manually** as input features. 

For example: add sunshine hours from the last 30 days as separate columns. 



In [ ]:
# Create lagged features for all Basel-specific predictors
data_lagged = data.copy()
basel_columns = [col for col in data.columns if col.startswith('BASEL_')]

# Add lags for each Basel predictor (store in dictionary first)
window_size = 30
lagged_features = {}

for col in basel_columns:
    for lag in range(1, window_size + 1):
        lagged_features[f'{col}_lag{lag}'] = data[col].shift(lag)

# Concatenate all lagged features at once
data_lagged = pd.concat([data_lagged, pd.DataFrame(lagged_features)], axis=1)

# Drop rows with NaNs caused by lagging
data_lagged = data_lagged.dropna().reset_index(drop=True)

# Define X and y using only lagged Basel features
all_lagged_cols = [col for col in data_lagged.columns if col.startswith('BASEL_') and 'lag' in col]
X_data_lagged = data_lagged.loc[:nr_rows, all_lagged_cols]
y_data_lagged = data_lagged.loc[:nr_rows, 'BASEL_sunshine']  # target is same as before

# Train/validation/test split (preserving time order)
from sklearn.model_selection import train_test_split

X_train_lagged, X_holdout_lagged, y_train_lagged, y_holdout_lagged = train_test_split(
    X_data_lagged, y_data_lagged, test_size=test_set_size, random_state=0, shuffle=False)

X_val_lagged, X_test_lagged, y_val_lagged, y_test_lagged = train_test_split(
    X_holdout_lagged, y_holdout_lagged, test_size=0.5, random_state=0, shuffle=False)


In [ ]:
X_train_lagged.head()

In [ ]:
print(X_train_lagged.shape)
print(X_test_lagged.shape)
print(X_val_lagged.shape)


**Note**: Number of predictors grows drastically! How do you think this will impact model performance? Let's try it and see.

In [ ]:
# Create new dense model with additional input features
model_dense_lagged = create_dense_nn(input_shape=(X_train_lagged.shape[1],))

# view model summary
model_dense_lagged.summary()


**Note**: With 270 input features, the total number of weights increases dramaticaly as well. 32,301 (30 lags) >> 6,101 (1 lag)

Compile and train the model next.

In [ ]:
# compile model 
compile_model(model_dense_lagged)
# train model and store results
history_dense_lagged = model_dense_lagged.fit(X_train_lagged, y_train_lagged,
                    batch_size = 32,
                    epochs = 200,
                    validation_data=(X_val_lagged, y_val_lagged),
                    callbacks=[earlystopper])


In [ ]:
plot_history(history_dense_lagged, ['root_mean_squared_error', 'val_root_mean_squared_error'])

In [ ]:
model_dense_lagged.evaluate(X_test_lagged, y_test_lagged)

Here, we see that the explosion in input parameters has made the problem more challenging to model. We get a test set error of 3.96 compared to 3.59 in our previous dense model.

### Discuss: What might be happening here?

### Why manual lagging isn’t enough

To give our models access to past information, we previously added lagged versions of each predictor. This approach works — but it has several limitations:

- **No sense of time**: The model treats lagged inputs as unordered. It doesn’t know that lag 1 is closer to the present than lag 30.
- **Feature explosion**: The number of input features grows quickly with the window size. With 30 lags across 9 features, we ended up with 270 inputs.
- **Manual design choices**: We have to choose which lags and predictors to include, which introduces rigidity and may not generalize well.
- **Separate weights for each lag**: A dense model learns different weights for each timestep, making it hard to capture shifting or repeating patterns over time.

In short, manually lagging inputs increases model complexity and overfitting risk without offering the model a structured way to reason about time.

### Introducing recurrence

Recurrent neural networks (RNNs) offer a more efficient way to model temporal data. Instead of flattening the past into a wide array of lagged features, RNNs process sequences step by step — maintaining a **hidden state** that evolves over time.

At each timestep `t`, the RNN sees:
- the current input `x_t`, and
- the previous hidden state `h_{t-1}`, which summarizes all earlier inputs

This update looks like:



In [ ]:
#           ┌────────────┐
# x_t ───►  │  RNN cell  │ ───►  h_t
#           └────────────┘
#             ▲       
#         h_{t-1}     

### Why RNNs don't need lagged features
You can think of the hidden state h_(t) in an RNN as a compressed, learned summary or memory of everything the model has seen so far — not just one specific lag like x_{t-1}, but a running representation of the entire sequence up to that point. RNNs effectively create a *learned memory* of the past. They resemble a first-order Markov process in that each hidden state depends only on the previous one — but unlike traditional Markov models, the state is learned and far more expressive.

This shared-memory approach:
- makes the model more efficient: the **same weights** are used at every timestep, keeping parameter count low
- reduces the risk of overfitting, and
- the model doesn't need to be handed 30 explicit lags — it **learns what to remember** over time.
- allows model to flexibly capture patterns at multiple time scales.

In short: instead of manually handing over a huge chunk of history, we let the model carry and refine a compact representation of the past over time.

### A simple recurrent model
Recurrent layers expect 3D inputs: one sequence per sample. Instead of flattening the window (30 days) into 1 wide row, we reshape our data into a sequence of timesteps, each with a vector of predictors. At each time step t, the RNN takes:
- the input at this step x_t
- the hidden state from only the previous step h_{t-1}

It produces a new hidden state h_t, which is used for the next time step and sometimes for prediction.

#### Preparing the inputs and outputs
To use a recurrent model, we'll restructure our dataset so that each sample is a short sequence of consecutive days — each with the same set of predictors. Recurrent layers expect 3D input tensors:  
**(samples, timesteps, features per timestep)**

We'll start from the original data and build these sequences explicitly.

In [ ]:
X_data = data.loc[:nr_rows].drop(columns=["DATE", "MONTH"]) # nr_rows gives us 3 years of data again
y_data = data.loc[window_size:(nr_rows + window_size)]["BASEL_sunshine"] # predict starting with 8th day
X_data.shape

In [ ]:
# Keep only BASEL predictors 
basel_columns = [col for col in data.columns if col.startswith('BASEL_')]

# Drop NaNs and reset index to keep it tidy
data_basel = data[basel_columns].dropna().reset_index(drop=True)


#### How are the input sequences and targets constructed?

Each input sample is a sequence of 30 consecutive days of predictor values.  
We slide this 30-day window over the dataset to create many such sequences.


In [ ]:
import numpy as np

# Build sequences
n_seq = len(data_basel) - window_size

X_seq = np.stack([
    data_basel[basel_columns].iloc[i:i+window_size].values
    for i in range(n_seq)
])
print("X_seq shape:", X_seq.shape)  # (samples, 30, features)



This creates a 3D array of shape (samples, timesteps=30, features). Each slice `X_seq[i]` contains predictors for days i through i+window_size

For each input sequence, we want the model to predict the sunshine on the day after the last timestep (i.e., day i+window_size). We align the target values accordingly:

In [ ]:
y_seq = data_basel['BASEL_sunshine'].iloc[window_size:].values

print("y_seq shape:", y_seq.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_holdout, y_train, y_holdout = train_test_split(X_seq, y_seq, test_size=test_set_size, random_state=0, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, test_size=0.5, random_state=0, shuffle=False)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

### Modeling sequences with a recurrent neural network

Now that our input data is structured as sequences (30 timesteps × 89 features), we can use a recurrent neural network to learn from patterns over time.

Instead of manually adding lagged features, we feed the model the full sequence and let it learn which parts of the past are useful. This keeps the input compact while enabling the model to capture temporal dependencies internally.

Let's define our first `SimpleRNN` model using the same Keras Functional API as before.


In [ ]:

def create_simple_rnn(input_shape, rnn_units=16):
    # Input layer for sequences of shape (timesteps, features)
    inputs = keras.Input(shape=input_shape, name='input_sequence')

    # Simple RNN layer with ReLU activation
    x = keras.layers.SimpleRNN(rnn_units, activation='relu')(inputs)

    # Output layer for regression
    outputs = keras.layers.Dense(1)(x)

    return keras.Model(inputs=inputs, outputs=outputs, name="simple_rnn_weather_model")


In [ ]:
model_rnn = create_simple_rnn(input_shape=X_train.shape[1:], rnn_units=16)
model_rnn.summary()



The SimpleRNN layer has three components contributing to its parameter count of 416

- **Input weights (`W`)**:  
  One weight per input feature × hidden unit  
  → `9 input features × 16 units = 144`

- **Recurrent weights (`U`)**:  
  One weight per hidden unit × hidden unit  
  → `16 units × 16 units = 256`

  These weights connect the hidden state at time `t-1` to the hidden state at time `t`, enabling the model to "remember" and update internal state across timesteps.

- **Biases (`b`)**:  
  One bias term per hidden unit  
  → `16`

**Total for RNN**:  


In [ ]:
144 + 256 + 16

These weights are **shared across all 30 timesteps** — the same weights are applied as the model "unrolls" across time. This is one of the reasons RNNs can model long sequences without requiring separate weights for each lag like a dense model would.


In comparison, our dense model containing 30 lags with all predictors had 32,201 weights!

### What it means to "share weights" in an RNN

In a standard feedforward neural net, each feature connects to a unique set of weights. When we added lagged features, that meant each lag had its own distinct weights.

But in an RNN, we loop through time — and at each step, the same weight matrices are used:
- One set connects the input at time `t` to the hidden state 
- Another connects the previous hidden state to the next 
- A bias vector is added (`b`)

These shared weights are trained to work across *all* timesteps. That’s what allows an RNN to model temporal dependencies **without needing a separate set of weights for each lag**.

This is why RNNs are often more parameter-efficient than wide dense models that treat time as flat.


### What does the Dense layer do here?

After the RNN processes the full sequence, it returns the **final hidden state** — not the full sequence of outputs, just the one at the **last timestep**. This is a single vector of size 32 (one value per hidden unit).

That vector is then passed to the `Dense` layer, which:

- Takes all 16 hidden units from the final timestep
- Learns one weight per unit, to map them to a single prediction (sunshine hours)
- Adds a bias term

So the 17 parameters in the dense layer are:

- **16 weights**: One from each hidden unit to the output
- **1 bias**: Shifts the final output up or down

 Even though the RNN saw `window_size` days of data, **only the final hidden state is passed forward** to make the prediction. This is typical in many-to-one sequence models (e.g. predict tomorrow from the last 30 days).


### Compile and fit the model

In [ ]:
compile_model(model_rnn)

In [ ]:
history_rnn = model_rnn.fit(
    X_train, y_train,
    batch_size=32,
    epochs=200,
    validation_data=(X_val, y_val),
    callbacks=[earlystopper]
)

In [ ]:
plot_history(history_rnn, ['root_mean_squared_error', 'val_root_mean_squared_error'])


In [ ]:
model_rnn.evaluate(X_test, y_test)

We see an improvement with the RNN model! Our test error is now 3.487 compared to the dense net's error of 3.96. Let's try a slighlty more complicated RNN. This time, we'll use two recurrent layers

The first SimpleRNN layer reads the input sequence and returns a sequence of hidden states — one for each timestep -> Output shape: (batch_size, timesteps, units)
This preserves temporal information across all steps.

The second SimpleRNN layer then treats that sequence as its input, processing it step by step and finally returning a single hidden state — the one from the final timestep.
Output shape: (batch_size, units)

In [ ]:
from tensorflow import keras

def create_stacked_rnn_model(input_shape, rnn_units=16):
    inputs = keras.Input(shape=input_shape, name="input_sequence")

    x = keras.layers.SimpleRNN(rnn_units, return_sequences=True)(inputs) 
    x = keras.layers.SimpleRNN(rnn_units)(x) 
    outputs = keras.layers.Dense(1)(x)

    return keras.Model(inputs, outputs, name="stacked_rnn_weather_model")



In [ ]:
model_rnn_stacked = create_stacked_rnn_model(input_shape=X_train.shape[1:], rnn_units=16)
model_rnn_stacked.summary()
compile_model(model_rnn_stacked)
history_rnn_stacked = model_rnn_stacked.fit(
    X_train, y_train,
    batch_size=64,
    epochs=200,
    validation_data=(X_val, y_val),
    callbacks=[earlystopper]
)

In [ ]:
plot_history(history_rnn_stacked, ['root_mean_squared_error', 'val_root_mean_squared_error'])


In [ ]:
model_rnn_stacked.evaluate(X_test, y_test)

Further improvement! Yay recurrence!

## The problem with vanilla RNNs

Basic RNNs can capture short-term dependencies, but they struggle to retain information across long sequences — a limitation known as the vanishing gradient problem.

Imagine trying to predict the next word in a sentence:

> I grew up in France… I speak fluent ___.

You want the model to remember "France" — even if it happened many steps earlier. Vanilla RNNs often forget these long-range dependencies.



## LSTM to the rescue

LSTM (Long Short-Term Memory) layers address this by adding a memory component: the cell state.

In [ ]:
#           ┌────────────┐
# x_t ───►  │  LSTM cell │ ───►   h_t
#           └────────────┘
#             ▲       ▲
#         h_{t-1}   c_{t-1} (memory)

At each time step t, the LSTM takes:
- the input x_t
- the previous hidden state h_{t-1}
- the previous cell state c_{t-1}

The cell state acts as long-term memory, while the hidden state provides a short-term summary. Gates inside the LSTM control how much information to forget, store, or expose.

- **Forget gate**: What information should be erased from memory?
- **Input gate**: What new information should be stored?
- **Output gate**: What part of the memory should be passed forward?

This lets the model maintain a persistent internal state across many steps.


Train a basic LSTM model:

In [ ]:
def create_lstm_model(input_shape, lstm_units=16):
    inputs = keras.Input(shape=input_shape, name="input_sequence")

    # Stacked LSTM layers to compared to stacked RNN
    x = keras.layers.LSTM(lstm_units, return_sequences=True)(inputs) 
    x = keras.layers.LSTM(lstm_units)(x) 

    # Output layer
    outputs = keras.layers.Dense(1)(x)

    return keras.Model(inputs, outputs, name="lstm_weather_model")


In [ ]:
model_lstm = create_lstm_model(input_shape=X_train.shape[1:], lstm_units=16)
model_lstm.summary()
compile_model(model_lstm)
history_lstm = model_lstm.fit(
    X_train, y_train,
    batch_size=64,
    epochs=200,
    validation_data=(X_val, y_val),
    callbacks=[earlystopper]
)

The first LSTM layer (`lstm_7`) has four sets of parameters, one for each of its internal gates (input, forget, cell, output). Each set includes input weights, recurrent weights, and a bias term — so the total parameter count is scaled by a factor of 4.

### Breakdown of parameters in `lstm_7` (1,664 total):

- **Input weights (`W`)**:  
  One weight per input feature × hidden unit × 4 gates  
  → `9 input features × 16 units × 4 = 576`

- **Recurrent weights (`U`)**:  
  One weight per hidden unit × hidden unit × 4 gates  
  → `16 units × 16 units × 4 = 1,024`

- **Biases (`b`)**:  
  One bias term per unit × 4 gates  
  → `16 units × 4 = 64`

These weights are **shared across all timesteps** in the input sequence, which helps the model learn how to update its memory and output at each time step without growing the parameter count linearly with the sequence length.

**Total for `lstm_7`**:  
576 (input weights) + 1,024 (recurrent weights) + 64 (biases) = **1,664 parameters**


In [ ]:
plot_history(history_lstm, ['root_mean_squared_error', 'val_root_mean_squared_error'])


In [ ]:
model_lstm.evaluate(X_test, y_test)

Let's try a larger window length of 90 days. The LSTM model should be able to capture longer-range dependencies. 

In [ ]:
import numpy as np
window_size_larger = 90

# Build sequences
n_seq = len(data_basel) - window_size_larger

X_seq = np.stack([
    data_basel[basel_columns].iloc[i:i+window_size_larger].values
    for i in range(n_seq)
])
print("X_seq shape:", X_seq.shape)  # (samples, 30, features)

y_seq = data_basel['BASEL_sunshine'].iloc[window_size_larger:].values

print("y_seq shape:", y_seq.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_holdout, y_train, y_holdout = train_test_split(X_seq, y_seq, test_size=test_set_size, random_state=0, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, test_size=0.5, random_state=0, shuffle=False)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
model_lstm = create_lstm_model(input_shape=X_train.shape[1:], lstm_units=16)
model_lstm.summary()
compile_model(model_lstm)
history_lstm = model_lstm.fit(
    X_train, y_train,
    batch_size=64,
    epochs=200,
    validation_data=(X_val, y_val),
    callbacks=[earlystopper]
)

In [ ]:
plot_history(history_lstm, ['root_mean_squared_error', 'val_root_mean_squared_error'])


In [ ]:
model_lstm.evaluate(X_test, y_test)

## Keypoints

- RNNs and LSTMs allow neural networks to process data step-by-step
- LSTMs retain long-term context using gated memory
- Sequence models are widely used in time series, language, and biology:::